In [1]:
!pip install /kaggle/input/gradcam-wheel/ttach-0.0.3-py3-none-any.whl
!pip install /kaggle/input/gradcam-wheel/grad_cam-1.5.5-py3-none-any.whl

Processing /kaggle/input/gradcam-wheel/ttach-0.0.3-py3-none-any.whl
Processing /kaggle/input/gradcam-wheel/grad_cam-1.5.5-py3-none-any.whl


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pandas as pd
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

In [3]:
from albumentations import (Compose, Normalize, Resize, RandomResizedCrop, HorizontalFlip, VerticalFlip, ShiftScaleRotate, Transpose)
from albumentations.pytorch import ToTensorV2
from collections import OrderedDict
import cv2
import math
from pytorch_grad_cam import GradCAMPlusPlus
from torchvision.models import resnext50_32x4d

/opt/conda/lib/python3.10/site-packages/albumentations/check_version.py:49: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'
OUTPUT_DIR = './'
SUB_FILE = '../input/cassava-leaf-disease-classification/sample_submission.csv'
FIVEFOLD = "/kaggle/input/cassava-5-fold-csv/validation_data.csv"

STAGE_ONE_MODEL = "/kaggle/input/finergradcam/pytorch/default/1/gan_v3_fold0_best_g_g.pth"
STAGE_TWO_MODEL = "/kaggle/input/finergradcam/pytorch/default/1/finergradcam_fold0_best_d_d.pth"

In [6]:
class CFG:
    print_freq=300
    num_workers = 4
    model_name = 'finergradcam'
    size = 512
    epochs = 8
    factor = 0.2
    patience = 5
    eps = 1e-6
    lr = 1e-4
    min_lr = 1e-6
    batch_size = 12
    weight_decay = 1e-6
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    seed = 42
    target_size = 5
    target_col = 'label'
    n_fold = 5
    trn_fold = [1,2,3,4,5]
    
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

def get_transforms(*, data):
    if data == 'train':
        return Compose([
            RandomResizedCrop((CFG.size, CFG.size), scale=(0.4, 1.0)),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
    elif data == 'valid':
       return Compose([
           Resize(CFG.size, CFG.size),
           Normalize(
               mean=[0.485, 0.456, 0.406],
               std=[0.229, 0.224, 0.225],
           ),
           ToTensorV2(),
       ])
        
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.labels = df['label'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TRAIN_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        label = torch.tensor(self.labels[idx]).long()
        return image, label

# class TestDataset(Dataset):
#     def __init__(self, df, transform=None):
#         self.df = df
#         self.file_names = df['image_id'].values
#         self.labels = df['label'].values
#         self.transform = transform
        
#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         file_name = self.file_names[idx]
#         file_path = f'{TRAIN_PATH}/{file_name}'
#         image = cv2.imread(file_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         if self.transform:
#             augmented = self.transform(image=image)
#             image = augmented['image']
#         label = torch.tensor(self.labels[idx]).long()
#         return image, label

class TestDataset2(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_path_id'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        image = cv2.imread(file_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
            
        return image

class CassavaDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.labels = self.labels_df['label'].values

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.labels_df.iloc[idx, 0])
        # image = Image.open(img_name)
        # image = np.array(image)
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # label = self.labels_df.iloc[idx, 1]
        label = torch.tensor(self.labels[idx]).long()
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image, label


class Generator(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.backbone = resnext50_32x4d(weights=None)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)

    def forward(self, x):
        logits = self.backbone(x)
        return logits

    def get_cam(self, x, class_idx=None):
        features = []
        gradients = []

        def forward_hook(module, input, output):
            features.append(output)

        def backward_hook(module, grad_in, grad_out):
            gradients.append(grad_out[0])

        handle_f = self.backbone.layer4.register_forward_hook(forward_hook)
        handle_b = self.backbone.layer4.register_full_backward_hook(backward_hook)

        logits = self.forward(x)
        if class_idx is None:
            class_idx = logits.argmax(dim=1)

        one_hot = torch.zeros_like(logits)
        for i in range(logits.size(0)):
            one_hot[i, class_idx[i]] = 1

        self.zero_grad()
        logits.backward(gradient=one_hot, retain_graph=True)

        cams = []
        for i in range(x.size(0)):
            grad = gradients[0][i]
            feat = features[0][i]
            weights = grad.mean(dim=(1, 2))
            cam = torch.sum(weights.view(-1, 1, 1) * feat, dim=0)
            cam = F.relu(cam)
            cam = cam - cam.min()
            cam = cam / (cam.max() + 1e-8)
            cams.append(cam.unsqueeze(0))

        cam_tensor = torch.stack(cams)
        cam_tensor = F.interpolate(cam_tensor, size=(x.size(2), x.size(3)), mode='bilinear', align_corners=False)
        handle_f.remove()
        handle_b.remove()
        return cam_tensor.detach()

class Discriminator(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        base = resnext50_32x4d(weights=None)
        base.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.feature_extractor = nn.Sequential(
            base.conv1,
            base.bn1,
            base.relu,
            base.maxpool,
            base.layer1,
            base.layer2,
            base.layer3,
            base.layer4,
            base.avgpool,
        )
        self.flatten = nn.Flatten()
        in_features = base.fc.in_features
        self.fc_class = nn.Linear(in_features, num_classes)
        # self.fc_judge = nn.Linear(in_features, 1)

    def forward(self, image, cam):
        x = torch.cat([image, cam], dim=1)
        feat = self.feature_extractor(x)
        feat = self.flatten(feat)
        class_logits = self.fc_class(feat)
        # judge_logits = self.fc_judge(feat)
        # return class_logits, judge_logits
        return class_logits


In [7]:
class CustomTarget:
    def __init__(self, gt_class):
        self.gt_class = gt_class

    def __call__(self, model_output):
        # model_output: [num_classes]
        if self.gt_class == 4:
            return model_output[4]
        else:
            return model_output[self.gt_class] - 0.6 * model_output[4]

def remove_module_prefix(state_dict):
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        new_k = k.replace("backbone.", "")
        new_state_dict[new_k] = v
    return new_state_dict

In [8]:
class D1(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, row[0])
        # image = Image.open(img_path).convert("RGB")
        image = cv2.imread(img_path)  # row[0] is the image file path
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = int(row[1])

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return idx, image, label

val_transforms = Compose([
           Resize(CFG.size, CFG.size),
           Normalize(
               mean=[0.485, 0.456, 0.406],
               std=[0.229, 0.224, 0.225],
           ),
           ToTensorV2(),
       ])

In [9]:
if __name__ == '__main__':
    # test_df = pd.read_csv(SUB_FILE)
    
    # # test_dataset = TestDataset(transform=get_transforms(data='valid'))
    # train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
    # # Split into folds for cross validation - we used the same split for all the models we trained!
    # folds = train.merge(
    #     pd.read_csv(FIVEFOLD)[["image_id", "fold"]], on="image_id")
    # fold = 0
    # trn_idx = folds[folds['fold'] != fold].index
    # val_idx = folds[folds['fold'] == fold].index
    # train_folds = folds.loc[trn_idx].reset_index(drop=True)
    # valid_folds = folds.loc[val_idx].reset_index(drop=True)
    # valid_dataset = CassavaDataset(valid_folds, transform=get_transforms(data='valid'))
    # valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, 
    #                           shuffle=False, num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    # # load
    # generator = resnext50_32x4d(weights=None).cuda()
    # generator.fc = nn.Linear(generator.fc.in_features, 5)
    # state_dict = torch.load(STAGE_ONE_MODEL)['model']
    # state_dict = remove_module_prefix(state_dict)
    # generator.load_state_dict(state_dict)
    # generator = generator.cuda()
    # discriminator = Discriminator().cuda()
    # state_dict = torch.load(STAGE_TWO_MODEL)['model']
    # discriminator.load_state_dict(state_dict)
    # discriminator = discriminator.cuda()
    
    # criterion_class = nn.CrossEntropyLoss()
    # criterion_judge = nn.BCEWithLogitsLoss()

    # target_layers = [generator.layer4[-1]]
    # cam = GradCAMPlusPlus(model=generator, target_layers=target_layers)

    # generator.eval()
    # discriminator.train()
    # losses_d = AverageMeter()
    # losses_d_class = AverageMeter()
    # losses_d_judge = AverageMeter()
    # # === Validation === (simulate validation set)
    # losses_g = AverageMeter()
    # losses_d = AverageMeter()
    # generator.train()
    # discriminator.eval()
    # total_correct = 0
    # total_d_correct = 0
    # total_samples = 0

    # predictions = []
    # for step, (val_images, val_labels) in enumerate(valid_loader):
    #     batch_size = val_labels.size(0)
    #     val_images = val_images.cuda()
    #     val_labels = val_labels.cuda()

    #     # forward pass for generator with gradient tracking
    #     val_logits = generator(val_images)
    #     val_preds = val_logits.argmax(dim=1)
    #     val_correct = (val_preds == val_labels)

    #     val_cams = []
    #     for i in range(batch_size):
    #         input_tensor = val_images[i].unsqueeze(0)
    #         gt = val_labels[i].item()
    #         target = [CustomTarget(gt)]
    #         grayscale_cam = cam(input_tensor=input_tensor, targets=target)[0]
    #         val_cams.append(grayscale_cam)

    #     val_cams = torch.from_numpy(np.stack(val_cams)).unsqueeze(1).float().cuda()
  
    #     with torch.no_grad():
    #         val_g_loss = criterion_class(val_logits, val_labels)
    #         val_d_class_logits = discriminator(val_images, val_cams)
    #         val_d_class_loss = criterion_class(val_d_class_logits, val_labels)
    #         val_d_preds = val_d_class_logits.argmax(dim=1)

    #         predictions.extend(val_d_preds.cpu().numpy())  # Store predictions
            
    #         val_judge_labels = (val_preds == val_labels).float().unsqueeze(1)
    #         val_d_loss = val_d_class_loss
    #         val_d_correct = (val_d_preds == val_labels)

    #     losses_g.update(val_g_loss.item(), batch_size)
    #     losses_d.update(val_d_loss.item(), batch_size)
    #     total_correct += val_correct.sum().item()
    #     total_d_correct += val_d_correct.sum().item()
    #     total_samples += batch_size
    
    # epoch_val_acc = total_correct / total_samples
    # epoch_val_d_acc = total_d_correct / total_samples
    # print(f"Val G Accuracy: {epoch_val_acc:.4f} | Val D Accuracy: {epoch_val_d_acc:.4f}")
    # print(f"Gen Loss: {losses_g.avg:.4f} | Dis Loss: {losses_d.avg:.4f}")

    # predictions = np.array(predictions)
    # test_df['label'] = predictions
    # test_df.to_csv('submission.csv', index=False)
    # print("Submission file created!")
    ###########################################################################################
    test_df = pd.read_csv(SUB_FILE)
    
    # test_dataset = TestDataset(transform=get_transforms(data='valid'))
    # test_dataset = CassavaDataset(SUB_FILE, TEST_PATH, transform=get_transforms(data='valid'))
    # predictions_resnext = pd.DataFrame(columns={"image_id"})
    # predictions_resnext["image_id"] = submission_df["image_id"].values
    # predictions_resnext['image_path_id'] = image_path + predictions_resnext['image_id'].astype(str)
    # test_dataset = TestDataset2(predictions_resnext, transform=get_transforms(data='valid'))

    # test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, 
    #                           shuffle=False, num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    test_df = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
    test_dataset = D1('../input/cassava-leaf-disease-classification/sample_submission.csv',
                      '../input/cassava-leaf-disease-classification/test_images',
                      transform=val_transforms)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
    
    # load
    generator = resnext50_32x4d(weights=None).cuda()
    generator.fc = nn.Linear(generator.fc.in_features, 5)
    state_dict = torch.load(STAGE_ONE_MODEL)['model']
    state_dict = remove_module_prefix(state_dict)
    generator.load_state_dict(state_dict)
    generator = generator.cuda()
    discriminator = Discriminator().cuda()
    state_dict = torch.load(STAGE_TWO_MODEL)['model']
    discriminator.load_state_dict(state_dict)
    discriminator = discriminator.cuda()
    
    criterion_class = nn.CrossEntropyLoss()
    criterion_judge = nn.BCEWithLogitsLoss()

    target_layers = [generator.layer4[-1]]
    cam = GradCAMPlusPlus(model=generator, target_layers=target_layers)

    generator.eval()
    discriminator.train()
    # === Validation === (simulate validation set)
    generator.train()
    discriminator.eval()
    total_correct = 0
    total_d_correct = 0
    total_samples = 0

    predictions = []
    # for step, (val_images, val_labels) in enumerate(test_loader):
    for _, val_images, _ in test_loader:
        batch_size = val_images.size(0)
        val_images = val_images.cuda()

        # forward pass for generator with gradient tracking
        val_logits = generator(val_images)
        val_preds = val_logits.argmax(dim=1)

        val_cams = []
        for i in range(batch_size):
            input_tensor = val_images[i].unsqueeze(0)
            target = [CustomTarget(val_preds[i].item())]
            grayscale_cam = cam(input_tensor=input_tensor, targets=target)[0]
            val_cams.append(grayscale_cam)

        val_cams = torch.from_numpy(np.stack(val_cams)).unsqueeze(1).float().cuda()
  
        with torch.no_grad():
            val_d_class_logits = discriminator(val_images, val_cams)
            # val_d_preds = val_d_class_logits.argmax(dim=1)
            _, val_d_preds = torch.max(val_d_class_logits, 1)
            predictions.extend(val_d_preds.cpu().numpy())  # Store predictions

    predictions = np.array(predictions)
    test_df['label'] = predictions
    test_df.to_csv('submission.csv', index=False)
    print("Submission file created!")

/tmp/ipykernel_24/1873441109.py:120: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(STAGE_ONE_MODEL)['model']
/tmp/ipykernel_24/1873441109.py:125: Fut

Submission file created!
